# Vineyards Navigation with Semantic Segmentation - All Crops

Using semantic segmentation for a proportional controller along the vineyard rows.




- This notebook contains scripts to create, train, and test a deep learning network to perform fast semantic segmentation on platform with mobile CPUs and low memory capabilities

- The implemented architecture is a MobileNetV3 with a customized LR-ASPP

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%cfg Completer.use_jedi = False

In [ ]:
# Libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

import glob
import math
import random
from pathlib import Path

#import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as tqdm

import tensorflow as tf
import tensorflow_addons as tfa

from utils.tools import *
from utils.preprocess import *
from utils.visualize import *
from utils.training_tools import *
from utils.models import build_model_multi, build_model_binary
from utils.cityscapes_utils import CityscapesDataset
from utils.mobilenet_v3 import MobileNetV3Large 

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", physical_devices)

#select the working GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_memory_growth(gpus[0], True)
#mirrored_strategy = tf.distribute.MirroredStrategy(devices=['GPU:0','GPU:1'])

In [ ]:
# define dataset import paths
ROOT = Path('datasets/')
CROPS = ['dataset_lattuga', 'dataset_zucchini', 'dataset_bieta', 'dataset_pero']

DATASETS = sorted([ROOT.joinpath(d) for d in CROPS])

In [ ]:
DATASETS

In [ ]:
# define some variables and read cfg
config_path = 'utils/cfg.yaml'
model_dir = Path('bin/')
logs_dir = Path('logs')
cfg = read_yaml(config_path)

cfg

# Pre-process the dataset

In [ ]:
def load_subdataset(root, cfg):
    img_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory=root.joinpath('images'),
        labels=None,
        label_mode=None,
        class_names=None,
        color_mode="rgb",
        batch_size=1, # cannot set None in TF 2.6!
        image_size=(cfg['IMG_SIZE'], cfg['IMG_SIZE']),
        shuffle=False,
        seed=None,
        interpolation="bilinear",
        follow_links=False)

    img_ds = img_ds.map(normalize_imagenet)

    mask_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory=root.joinpath('masks'),
        labels=None,
        label_mode=None,
        class_names=None,
        color_mode="grayscale",
        batch_size=1, # cannot set None in TF 2.6!
        image_size=(cfg['IMG_SIZE'], cfg['IMG_SIZE']),
        shuffle=False,
        seed=None,
        interpolation="bilinear",
        follow_links=False)
    
    mask_ds = mask_ds.map(normalize)
    
    return tf.data.Dataset.zip((img_ds, mask_ds))

In [ ]:
def load_dataset(root, cfg):
    for f in sorted([root.joinpath(d) for d in os.listdir(root) if not d.startswith('.')]):
        print(f)
        if 'ds' in locals():
            ds = ds.concatenate(load_subdataset(f, cfg))
        else:
            ds = load_subdataset(f, cfg)
    return ds

In [ ]:
def load_datasets(datasets, cfg):
    for crop in datasets:
        if 'ds' in locals():
            ds = ds.concatenate(load_dataset(crop, cfg))
        else:
            ds = load_dataset(crop, cfg)
    return ds

In [ ]:
ds = load_datasets(DATASETS, cfg)

In [ ]:
ds_len = len(ds)
test_len = math.floor(ds_len*0.2)
val_len = math.floor(test_len*0.2)
train_len = ds_len - test_len - val_len

In [ ]:
ds = ds.unbatch()
ds = ds.shuffle(ds_len)

ds_test = ds.take(val_len)
ds_train = ds.skip(val_len)

ds_val = ds_train.take(val_len)
ds_train = ds_train.skip(val_len)

In [ ]:
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(train_len)
ds_train = ds_train.map(data_aug)
ds_train = ds_train.batch(cfg['BATCH_SIZE'])
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_val = ds_val.cache()
ds_val = ds_val.batch(cfg['BATCH_SIZE'])
ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.cache()
ds_test = ds_test.batch(cfg['BATCH_SIZE'])
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# VISUALIZE

In [ ]:
n_images = 1

for img, gt in list(ds_test.unbatch().take(n_images)):
    visualize_images(img, gt, pred=None, denormalize=True, vineyard=True, imagenet=True)

# Create Mobilenet-v3 with LR-ASPP

In [ ]:
#base model mobile net

#with mirrored_strategy.scope():
backbone = MobileNetV3Large(input_shape=(cfg['IMG_SIZE'], cfg['IMG_SIZE'], 3),
                            alpha=1.0,
                            minimalistic=False,
                            include_top=False,
                            weights='imagenet',
                            input_tensor=None,
                            classes=cfg['N_CLASSES'],
                            pooling='avg',
                            dropout_rate=0,
                            backend=tf.keras.backend, 
                            layers=tf.keras.layers, 
                            models=tf.keras.models, 
                            utils=tf.keras.utils)

model = build_model_binary(backbone, 0, 1)

In [ ]:
model.summary()

# Train the model

In [ ]:
name_model = 'lr_aspp_multicrop_sgd8_b'+str(cfg['BATCH_SIZE'])+'_binary_cityscapes.h5'

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath=model_dir, 
                               monitor = 'val_loss',
                               verbose=1, 
                               save_best_only=True)

total_update_steps = cfg['N_EPOCHS'] * (train_len) // cfg['BATCH_SIZE']

lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=1e-3, 
                                                            decay_steps=total_update_steps, 
                                                            end_learning_rate=1e-6, power=0.96)

In [ ]:
adam_w = tfa.optimizers.AdamW(learning_rate=lr_schedule, weight_decay=1e-5)
sgd = tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.99, nesterov=False)
adam = tf.keras.optimizers.Adam(learning_rate=3e-4)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss=tf.losses.BinaryCrossentropy(),
              metrics = [mIoU])

In [ ]:
# train the model on the new data for a few epochs
history = model.fit(ds_train, validation_data=ds_val,
                    epochs=cfg['N_EPOCHS'],
                    callbacks=[])  

In [ ]:
plot_history(history, metric='metric_IoU')

In [ ]:
model.save(model_dir.joinpath(name_model))

# Test the Model

## Test on test set with training metrics

In [ ]:
name_model = 'multicrop_pero_KD.h5'
model.load_weights(model_dir.joinpath(name_model))

In [ ]:
model.evaluate(ds_test)

## Test on some samples

In [ ]:
n_images = 5

for img, gt in list(ds_test.unbatch().take(n_images)):
    #print(img.shape)
    #print(gt)
    pred = predict_model_binary(model, img, 0.5).astype(float)
    visualize_images(img, gt, pred[0,...,0], denormalize=True, vineyard=True, imagenet=True)

# Optimize and save tflite model

In [ ]:
raise

In [ ]:
model_path = model_dir.joinpath(name_model)

In [ ]:
model_path

In [ ]:
from utils.mobilenet_v3 import *

model.load_weights(str(model_path))
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("segmentation.tflite", "wb").write(tflite_model)

In [ ]:
#define functions to import and use tflite model
def input_tensor(interpreter):
    """Returns input tensor view as numpy array of shape (height, width, 3)."""
    tensor_index = interpreter.get_input_details()[0]['index']
    return interpreter.tensor(tensor_index)()[0]

def output_tensor(interpreter):
    """Returns dequantized output tensor."""
    output_details = interpreter.get_output_details()[0]
    output_data = interpreter.tensor(output_details['index'])()
    return output_data

def setInput(interpreter, data):
    """Copies data to input tensor."""
    input_tensor(interpreter)[:, :] = data

In [ ]:
interpreter = tf.lite.Interpreter('segmentation.tflite')
interpreter.allocate_tensors()

In [ ]:
n_images = 10

for img, gt in list(ds_test.unbatch().take(n_images)):
    #print(img.shape)
    #print(gt.shape)
    #model input
    setInput(interpreter, img[None,...])
    # invoke interpreter
    interpreter.invoke()
    y_pred = output_tensor(interpreter)[0]
    y_pred = (y_pred > 0.5)
    #print(y_pred.shape)
    visualize_images(img, gt, y_pred, denormalize=False, vineyard=True)

# Test

In [ ]:
import h5py

In [ ]:
model.load_weights('bin/multicrop_baseline_zucchini_None.h5')

In [ ]:
model.weights[0]

In [ ]:
model.load_weights('bin/multicrop_miou_zucchini_PADAIN.h5')

In [ ]:
model.weights[0]

# Contrastive Loss

In [ ]:
import tensorflow as tf
# from utils.training_tools import ContrastiveLoss
from utils.models import build_model_binary
from utils.mobilenet_v3 import MobileNetV3Large 

In [ ]:
BS = 8

In [ ]:
backbone = MobileNetV3Large(input_shape=(224, 224, 3),
                            alpha=1.0,
                            minimalistic=False,
                            include_top=False,
                            weights=None,
                            input_tensor=None,
                            classes=1,
                            pooling='avg',
                            dropout_rate=False,
                            inst_norm='ISW', 
                            p=0,
                            eps=1e-5,
                            whiten_layers=[0,1,2],
                            backend=tf.keras.backend, layers=tf.keras.layers, models=tf.keras.models, 
                            utils=tf.keras.utils)

In [ ]:
backbone.summary()

In [ ]:
ls = ['instance_normalization',
      'instance_normalization_1',
      'instance_normalization_2']

In [ ]:
print(backbone.get_layer('Conv').weights)

In [ ]:
print(backbone.get_layer('Conv').weights)

In [ ]:
print(backbone.get_layer('Conv').weights)

In [ ]:
for l in ls:
    print(backbone.get_layer(l).output)

In [ ]:
model = build_model_binary(backbone, 0.2, 1, dg='PADAIN')

In [ ]:
model.summary()

In [ ]:
inp = tf.random.uniform((BS,224,224,3))
y, f = model(inp, training=True)
y_b, f_b = model(inp, training=False)

In [ ]:
y.shape, f.shape
y_b.shape, f_b.shape

In [ ]:
class ContrastiveLoss(tf.keras.losses.Loss):
    """
    Constrastive loss for features-matching clustering ispired by ArXiv:2002.05709
    """
    def __init__(self, batch_size, weight=0.1, name='cluster_loss'):
        super().__init__(name=name)
        self.contrastive_labels = tf.range(batch_size)
        self.weight = weight
        self.temperature = 0.1
        
    def call(self, y_pred, y_pred_):
        B, H, W, C = y_pred.shape
        y_pred = tf.reshape(y_pred, (B, H*W*C))
        y_pred_ = tf.reshape(y_pred_, (B, H*W*C))
        projections_1 = tf.math.l2_normalize(y_pred, axis=1)
        projections_2 = tf.math.l2_normalize(y_pred_, axis=1)
        similarities = (
            tf.matmul(projections_1, projections_2, transpose_b=True) / self.temperature
        )
        print(similarities.shape)
        # The temperature-scaled similarities are used as logits for cross-entropy
        # a symmetrized version of the loss is used here
        loss_1_2 = tf.keras.losses.sparse_categorical_crossentropy(
            self.contrastive_labels, similarities, from_logits=True
        )
        loss_2_1 = tf.keras.losses.sparse_categorical_crossentropy(
            self.contrastive_labels, tf.transpose(similarities), from_logits=True
        )
        return (loss_1_2 + loss_2_1) / 2 * self.weight

In [ ]:
cl = ContrastiveLoss(BS, weight=0.1)

In [ ]:
cl(f,f_b)

In [ ]:
print(f[0,...,0])

In [ ]:
print(f_b[0,...,0])

# Test Inference

In [ ]:
import tensorflow as tf
from pathlib import Path
# from utils.training_tools import ContrastiveLoss
from utils.models import build_model_binary
from utils.mobilenet_v3 import MobileNetV3Large 
import os
from PIL import Image 
import numpy as np
from utils.data import normalize_imagenet
import matplotlib.pyplot as plt

from utils.tools import read_yaml
from utils.data import load_multi_dataset

from utils.visualize import *


In [ ]:
#select CPU
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
%cfg Completer.use_jedi = False

In [ ]:
cfg = read_yaml('utils/cfg.yaml')
cfg['TARGET'] = 'albero'

In [ ]:
target_dataset = Path('datasets/').joinpath(f"dataset_{cfg['TARGET']}")

ds_source, ds_target = load_multi_dataset([target_dataset], target_dataset, cfg)

In [ ]:
ds_source

In [ ]:
name_model = 'baselines/multicrop_baseline_aug_lattuga_None.h5'
model.load_weights(model_dir.joinpath(name_model))

In [ ]:
n_images = 1

for img, gt in list(ds_source[0].unbatch().take(n_images)):
    print(np.min(img), np.max(img))
    visualize_images(img, gt, pred=None, denormalize=True, vineyard=True, imagenet=True)

In [ ]:
def predict_image(img):
    i = Image.open(img)
    i = np.array(i)    
    if i.shape[-1] > 3:
        i = i[...,:-1]
        
    i_sq = tf.image.resize(i,(224,224))
    i = tf.keras.applications.imagenet_utils.preprocess_input(i, mode='torch')
    i = tf.image.resize(i, [224, 224])
    out = model.predict(i[None,...])[0][0]
    
    plt.imshow(out, alpha=0.7)
    plt.imshow(i_sq/255.0, alpha=0.5) 
    plt.show()

In [ ]:
s = 0.5

ROOT_I = Path('datasets/dataset_bieta/dataset_bieta_1/images/Image1/')
LRI = [str(ROOT_I.joinpath(f)) for f in os.listdir(ROOT_I)]
    
for j in range(5):
    predict_image(LRI[j])
    break

In [ ]:
Image.open('datasets/test/lavanda/filare_largo/frame0.png')

In [ ]:
predict_image('datasets/test/lavanda/filare_largo/frame1800.png')

In [ ]:
img_list = [
    'datasets/dataset_zucchini/dataset_zucchini_1/images/Image1/Image0001.png',
    'datasets/dataset_bieta/dataset_bieta_1/images/Image1010/Image0001.png',
    'datasets/dataset_lattuga/dataset_lattuga_1/images/Image1010/Image0001.png',
    'datasets/dataset_pero/dataset_pero_1/images/Image1010/Image0001.png',
    'datasets/dataset_albero/dataset_albero_1/images/Image1/rgb0.png',
    'datasets/dataset_vite/dataset_vite_1/images/Image1/Image0001.png'
]

In [ ]:
for i in img_list:
    predict_image(i)

In [ ]:
i = Image.open('datasets/dataset_zucchini/zucchini_dataset2/images/Image1001/Image0001.png')

In [ ]:
i = np.array(i)

In [ ]:
plt.imshow(i)
plt.show()

In [ ]:
i_sq = tf.image.resize(i,(224,224))

In [ ]:
i = normalize_imagenet(i[...,:-1])

In [ ]:
plt.imshow(np.round(i).astype('uint8'))
plt.show()

In [ ]:
i = tf.image.resize(i, [224, 224])

In [ ]:
plt.imshow(np.round(i).astype('uint8'))
plt.show()

In [ ]:
out = model.predict(i[None,...])[0][0]

In [ ]:
out.shape

In [ ]:
plt.imshow(out, alpha=0.6)
plt.imshow(i_sq/255.0, alpha=0.5) 

# TFLite Conversion

In [ ]:
model.load_weights('bin/lavanda_some_albero_None.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()

name_model_tflite = 'zucchini.tflite'
tflite_model_file = Path(cfg['MODEL_PATH']).joinpath(name_model_tflite)                          
tflite_model_file.write_bytes(tflite_model)

# Test

In [ ]:
r = list(range(10))

In [ ]:
r

In [ ]:
r[::2]

In [ ]:
(12960+1440)/3

In [ ]:
kd_loss_fn = tf.keras.losses.KLDivergence(reduction=tf.keras.losses.Reduction.NONE)
T = 4

In [ ]:
pred_t = tf.random.uniform((32,224,224,1))*10
pred = tf.random.uniform((32,224,224,1))*10

In [ ]:
pred_t = tf.reshape(pred_t,(32,-1))
pred = tf.reshape(pred,(32,-1))

In [ ]:
aux_loss = kd_loss_fn(tf.nn.softmax(pred_t / T, axis=-1),
                      tf.nn.softmax(pred / T, axis=-1))* T**2

In [ ]:
aux_loss.shape

In [ ]:
tf.reduce_mean(aux_loss)